In [247]:
#!/usr/bin/env python3
"""
Scores include:
mCH CH mCH_rate mCH_fmr mCH_fumr mCH_mhl mCH_umhl 
mCG CG mCG_rate mCG_fmr mCG_fumr mCG_mhl mCG_umhl

fmr: fully methylated rate (read level)
fumr: fully unmethylated rate (read level)
mhl: methylation haplotype load (Gou et al 2017)
umhl: unmethylation haplotype load (symmetric to mhl)
"""

import numpy as np
from scipy import linalg
import pandas as pd
import argparse
import re

## Test genomewide

In [295]:
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')
# sys.path.insert(0, '/cndd2/fangming/projects/cfdna/MethHaplotypeLoad')
import pysam
import time
import snmcseq_utils
import compute_scores_utils
import logging
from multiprocessing import Pool

# import importlib
# importlib.reload(compute_scores_utils)

# # import "02.compute_scores"
# # import imp
# # compute_scores = imp.load_source('', '/cndd2/fangming/projects/cfdna/MethHaplotypeLoad/02.compute_scores.py')
# import importlib
# importlib.import_module('/cndd2/fangming/projects/cfdna/MethHaplotypeLoad/02.compute_scores.py')

In [251]:
# samfile.close()
# for read in samfile.fetch('chr1', 1000000, 1001000):
#     print(read)
#     break
# #     print(type(read))
# #     print(read)

### Proto 1 

In [68]:
# prototype1 - test basics on a small file
ti = time.time()

input_bam = '/cndd2/fangming/projects/cfdna/all_data/bam/test.sorted.bam'
output_mcinfo = '/cndd2/fangming/projects/cfdna/all_data/bam/test_output.mc_info.txt'
bin_size = 10000

# genome_sizes = snmcseq_utils.get_chrom_lengths_mouse()

bamfile = pysam.AlignmentFile(input_bam, 'rb')
print(genome_sizes)
    

current_chrom = "chr0"
current_binstart = 0
current_binend = current_binstart + bin_size 
current_binstring = ""

with open(output_mcinfo, 'w') as output_fh:
    for i, read in enumerate(bamfile.fetch()):
        chrom, start, end, mc_string = (read.reference_name, 
                                        read.reference_start, 
                                        read.reference_end, 
                                        read.tags[2][1],
                                       )
        # handle chr
        if not chrom.startswith('chr'):
            chrom = 'chr'+chrom
        
        # when current read goes beyond the current bin
        # keep updating the current bin until the current read is within the bin 
        while (current_chrom != chrom) or (start > current_binend):
            # flush the current bin out
            if current_binstring:
                current_binstring = current_binstring.replace('.', "")
                output_fh.write("{}\t{}\t{}\t{}\n".format(
                    current_chrom, current_binstart, current_binend, 
                    current_binstring))

            # move current bin to the next
            current_chrom = chrom
            current_binstring = ""
            current_binstart += bin_size
            current_binend += bin_size

        # move the current read the current bin
        current_binstring = current_binstring + (mc_string+',')
    

        
print("Total time: {:.2f}".format(time.time()-ti))

1     195471971
2     182113224
3     160039680
4     156508116
5     151834684
6     149736546
7     145441459
8     129401213
9     124595110
10    130694993
11    122082543
12    120129022
13    120421639
14    124902244
15    104043685
16     98207768
17     94987271
18     90702639
19     61431566
X     171031299
Name: 1, dtype: int64
Total time: 0.55


### Proto 2 

In [97]:
# prototype2 - test on a full bam file
# fixed bug when switching chrom
# 700 seconds to run, another 33 mins to calculate scores afterwards
# robust against bin size (varing bin size does not affect the time)

# Todo
# need to implement read split if it strides 2 bins
# try calculate scores on the fly 

ti = time.time()

input_bam = '/cndd2/fangming/projects/cfdna/all_data/bam/chan2013_CTR101.deduplicated.sorted.bam'
output_mcinfo = '/cndd2/fangming/projects/cfdna/all_data/bam/test_phase2.mc_info.txt'
bin_size = 100000 # 100,000
verbose_level = 1000000 # update every 1 million reads

# genome_sizes = snmcseq_utils.get_chrom_lengths_mouse()
# print(genome_sizes)
bamfile = pysam.AlignmentFile(input_bam, 'rb')

current_chrom = "chr0"
current_binstart = 0
current_binend = current_binstart + bin_size 
current_binstring = ""

with open(output_mcinfo, 'w') as output_fh:
    for i, read in enumerate(bamfile.fetch()):
        if i % verbose_level == 0:
            print(i, time.time()-ti)
            
        chrom, start, end, mc_string = (read.reference_name, 
                                        read.reference_start, 
                                        read.reference_end, 
                                        read.tags[2][1],
                                       )
        # handle chr
        if not chrom.startswith('chr'):
            chrom = 'chr'+chrom
        
        # when current read goes beyond the current bin
        # keep updating the current bin until the current read is within the bin 
        while (current_chrom != chrom) or (start > current_binend):
            # flush the current bin out
            if current_binstring:
                current_binstring = current_binstring.replace('.', "")
                output_fh.write("{}\t{}\t{}\t{}\n".format(
                    current_chrom, current_binstart, current_binend, 
                    current_binstring))

            # move current bin to the next 
            # (move to a new chrom or the next bin within the chrom)
            if current_chrom != chrom:
                current_chrom = chrom
                current_binstart = 0 
                current_binend = bin_size
                current_binstring = ""
            else:
                current_binstart += bin_size
                current_binend += bin_size
                current_binstring = ""

        # move the current read the current bin
        current_binstring = current_binstring + (mc_string+',')
    
bamfile.close()
print("Total time: {:.2f}".format(time.time()-ti))

0 0.04295086860656738
1000000 7.107336521148682
2000000 14.017687797546387
3000000 20.8909649848938
4000000 27.811792850494385
5000000 34.71662402153015
6000000 41.885241746902466
7000000 48.78362059593201
8000000 55.736910343170166


KeyboardInterrupt: 

### Proto 3 

In [239]:
# prototype3 - test calculating scores on the fly based on P2
# takes ~ 1hr to get the scores
# a pretty good version

# Todo
# could improve speeds if hold rows in memory?
# need to implement read split if it strides 2 bins

ti = time.time()

input_bam = '/cndd2/fangming/projects/cfdna/all_data/bam/chan2013_CTR101.deduplicated.sorted.bam'
output_mcinfo = '/cndd2/fangming/projects/cfdna/all_data/bam/test_phase3.mc_scores.txt'
bin_size = 100000 # 100,000
# verbose_level = 1000000 # update every 1 million reads
verbose_level = 100000 # update every 100,000 reads

columns = [
    'chr', 'start', 'end',
    'ch_mc', 'ch_c',
    'cg_mc', 'cg_c',
    'ch_fully_meth_reads', 'ch_fully_unmeth_reads', 'ch_total_reads',
    'cg_fully_meth_reads', 'cg_fully_unmeth_reads', 'cg_total_reads',
    'ch_mhl', 'ch_umhl',
    'cg_mhl', 'cg_umhl',
]


bamfile = pysam.AlignmentFile(input_bam, 'rb')

current_chrom = "chr0"
current_binstart = 0
current_binend = current_binstart + bin_size 
current_binstring = ""

with open(output_mcinfo, 'w') as output_fh:
    output_fh.write("\t".join(columns)+'\n')
    for i, read in enumerate(bamfile.fetch()):
        if i % verbose_level == 0:
            print(i, time.time()-ti)
            
        chrom, start, end, mc_string = (read.reference_name, 
                                        read.reference_start, 
                                        read.reference_end, 
                                        read.tags[2][1],
                                       )
        # handle chr
        if not chrom.startswith('chr'):
            chrom = 'chr'+chrom
        
        # when current read goes beyond the current bin
        # keep updating the current bin until the current read is within the bin 
        while (current_chrom != chrom) or (start > current_binend):
            # flush the current bin out
            if current_binstring:
                # compute all scores
                mch_str, mcg_str = compute_scores_utils.process_strings(current_binstring)
                # site level
                ch_mc, ch_c = compute_scores_utils.calc_mc(mch_str, 'h')
                cg_mc, cg_c = compute_scores_utils.calc_mc(mcg_str, 'z')
                # read level
                ch_fmr, ch_fumr, ch_total_reads = compute_scores_utils.calc_fmr(mch_str, 'h')
                cg_fmr, cg_fumr, cg_total_reads = compute_scores_utils.calc_fmr(mcg_str, 'z')
                # MHL level
                ch_mhl, ch_umhl = compute_scores_utils.calc_mhl(mch_str, 'h')
                cg_mhl, cg_umhl = compute_scores_utils.calc_mhl(mcg_str, 'z')
                
                column_values = [
                    chrom, start, end,
                    ch_mc, ch_c,
                    cg_mc, cg_c,
                    ch_fmr, ch_fumr, ch_total_reads,
                    cg_fmr, cg_fumr, cg_total_reads,
                    ch_mhl, ch_umhl,
                    cg_mhl, cg_umhl,
                ]
                output_fh.write("\t".join([str(item) for item in column_values])+'\n')
                
            
            # move current bin to the next 
            # (move to a new chrom or the next bin within the chrom)
            if current_chrom != chrom:
                current_chrom = chrom
                current_binstart = 0 
                current_binend = bin_size
                current_binstring = ""
            else:
                current_binstart += bin_size
                current_binend += bin_size
                current_binstring = ""

        # move the current read the current bin
        current_binstring = current_binstring + (mc_string+',')
    
bamfile.close()
print("Total time: {:.2f}".format(time.time()-ti))

0 0.02085733413696289
100000 3.774919033050537
200000 7.371954679489136
300000 10.448025703430176


KeyboardInterrupt: 

### Proto 4 

In [193]:
def calc_mhl_old(strings, letter):
    """MHL and uMHL
    """
    tfi = time.time()
    
    letter_lower = letter.lower()
    letter_upper = letter.upper()
    print("f1", time.time()-tfi)

    length_counts = np.bincount([len(string) for string in strings])
    length_counts_meth = np.zeros_like(length_counts)
    length_counts_unmeth = np.zeros_like(length_counts)
    print("f2", time.time()-tfi)

    for string in strings:
        # methylated
        a = re.split(r'{}+'.format(letter_lower), string.strip(letter_lower)) # get all substring including empty ones
        a = np.bincount([len(_a) for _a in a])
        length_counts_meth[:len(a)] += a

        # unmethylated
        a = re.split(r'{}+'.format(letter_upper), string.strip(letter_upper)) # get all substring including empty ones
        a = np.bincount([len(_a) for _a in a])
        length_counts_unmeth[:len(a)] += a
    print("f3", time.time()-tfi)
    
    length_counts_all = np.vstack([
        length_counts, 
        length_counts_meth, 
        length_counts_unmeth, 
        ])[:, 1:]
    print("f4", time.time()-tfi)
    
    dim = length_counts_all.shape[1]
    trans_mat = np.flip(linalg.hankel((np.arange(dim)+1)[::-1]), axis=0) # lower triangular matrix
    length_counts_all = np.dot(length_counts_all,trans_mat)
    print("f5", time.time()-tfi)

    fracs = length_counts_all[1:]/length_counts_all[0]
    weights = (np.arange(dim)+1)/(dim*(dim+1)/2)
    mhls = np.dot(fracs, weights)
    print("f6", time.time()-tfi)

    return mhls

In [229]:
# def calc_mhl_v1(strings, letter):
#     """MHL and uMHL
#     """
#     tfi = time.time()
    
#     letter_lower = letter.lower()
#     letter_upper = letter.upper()
#     print("f1", time.time()-tfi)

#     length_counts = np.bincount([len(string) for string in strings])
#     length_counts_meth = np.zeros_like(length_counts)
#     length_counts_unmeth = np.zeros_like(length_counts)
#     print("f2", time.time()-tfi)

#     # methylated
#     substrings = re.split(r'{}+'.format(letter_lower), letter_lower.join(strings)) # get all substring including empty ones
#     _a = np.bincount([len(string) for string in substrings])
#     length_counts_meth[:len(_a)] = _a
    
#     # unmethylated
#     substrings = re.split(r'{}+'.format(letter_upper), letter_upper.join(strings)) # get all substring including empty ones
#     _a = np.bincount([len(string) for string in substrings])
#     length_counts_unmeth[:len(_a)] = _a
#     print("f3", time.time()-tfi)
    
#     length_counts_all = np.vstack([
#         length_counts, 
#         length_counts_meth, 
#         length_counts_unmeth, 
#         ])[:, 1:]
#     print("f4", time.time()-tfi)
    
#     dim = length_counts_all.shape[1]
#     trans_mat = np.flip(linalg.hankel((np.arange(dim)+1)[::-1]), axis=0) # lower triangular matrix
#     length_counts_all = np.dot(length_counts_all,trans_mat)
#     print("f5", time.time()-tfi)

#     fracs = length_counts_all[1:]/length_counts_all[0]
#     weights = (np.arange(dim)+1)/(dim*(dim+1)/2)
#     mhls = np.dot(fracs, weights)
#     print("f6", time.time()-tfi)

#     return mhls

In [259]:
def calc_mhl_fast(strings, letter, verbose=False):
    """MHL and uMHL
    """
    if not strings:
        return [np.nan, np.nan]
    
    tfi = time.time()
    
    letter_lower = letter.lower()
    letter_upper = letter.upper()
    if verbose:
        print("f1", time.time()-tfi)

    length_counts = np.bincount([len(string) for string in strings])
    dim = len(length_counts) 
    if verbose:
        print("f2", time.time()-tfi)

    # methylated
    substrings_meth = re.split(r'{}+'.format(letter_lower), letter_lower.join(strings)) # get all substring including empty ones
    length_counts_meth = np.bincount([len(string) for string in substrings_meth], minlength=dim)
    
    # unmethylated
    substrings_unmeth = re.split(r'{}+'.format(letter_upper), letter_upper.join(strings)) # get all substring including empty ones
    length_counts_unmeth = np.bincount([len(string) for string in substrings_unmeth], minlength=dim)
    if verbose:
        print("f3", time.time()-tfi)
    
    length_counts_all = np.vstack([
        length_counts, 
        length_counts_meth, 
        length_counts_unmeth, 
        ])[:, 1:]
    if verbose:
        print("f4", time.time()-tfi)
    
    # transform matrix
    dim = dim - 1 # first entry - 0 length removed
    trans_mat = np.flip(linalg.hankel((np.arange(dim)+1)[::-1]), axis=0) # lower triangular matrix
    weights = (np.arange(dim)+1)/(dim*(dim+1)/2)
    
    # apply transformation 
    length_counts_all = np.dot(length_counts_all,trans_mat)
    fracs = length_counts_all[1:]/length_counts_all[0]
    mhls = np.dot(fracs, weights)
    
    if verbose:
        print("f6", time.time()-tfi)

    return mhls

In [262]:
# current_binstring
import pickle

strings_sample = 'mc_strings_sample.pkl'

# with open(strings_sample, 'wb') as fh:
#     pickle.dump((mch_str, mcg_str), fh)

with open(strings_sample, 'rb') as fh:
    mch_str, mcg_str = pickle.load(fh)
    

In [263]:
a = calc_mhl_old(mch_str, 'h')
print(a)
print('---')
a = calc_mhl_v1(mch_str, 'h')
print(a)
print('---')
a = calc_mhl_fast(mch_str, 'h', verbose=True)
print(a)

f1 2.6226043701171875e-06
f2 0.0003147125244140625
f3 0.013313055038452148
f4 0.013568401336669922
f5 0.01407623291015625
f6 0.015754222869873047
[  3.18627100e-06   9.90183831e-01]
---
f1 2.6226043701171875e-06
f2 0.0002429485321044922
f3 0.0008852481842041016
f4 0.0010733604431152344
f5 0.003155946731567383
f6 0.0035622119903564453
[  3.18627100e-06   9.90183831e-01]
---
f1 3.337860107421875e-06
f2 0.0002262592315673828
f3 0.0007047653198242188
f4 0.001825094223022461
f6 0.0025708675384521484
[  3.18627100e-06   9.90183831e-01]


In [264]:
a = calc_mhl_old(mcg_str, 'z')
print(a)
print('---')
a = calc_mhl_v1(mcg_str, 'z')
print(a)
print('---')
a = calc_mhl_fast(mcg_str, 'z', verbose=True)
print(a)

f1 2.86102294921875e-06
f2 0.0002682209014892578
f3 0.006798267364501953
f4 0.007059335708618164
f5 0.007292985916137695
f6 0.0074748992919921875
[ 0.40140556  0.01889466]
---
f1 1.3828277587890625e-05
f2 0.0002002716064453125
f3 0.0004742145538330078
f4 0.0005788803100585938
f5 0.0007834434509277344
f6 0.004652976989746094
[ 0.40140556  0.01889466]
---
f1 3.5762786865234375e-06
f2 0.0006005764007568359
f3 0.0008611679077148438
f4 0.0009665489196777344
f6 0.0012145042419433594
[ 0.40140556  0.01889466]


In [269]:
# prototype4 - improve speed with improved MHL calculation
# takes ~ 15 min to get the scores

# Todo
# need to implement read split if it strides 2 bins

ti = time.time()

input_bam = '/cndd2/fangming/projects/cfdna/all_data/bam/chan2013_CTR101.deduplicated.sorted.bam'
output_mcinfo = '/cndd2/fangming/projects/cfdna/all_data/bam/test_phase4.mc_scores.txt'
bin_size = 100000 # 100,000
verbose_level = 1000000 # update every 1 million reads

columns = [
    'chr', 'start', 'end',
    'ch_mc', 'ch_c',
    'cg_mc', 'cg_c',
    'ch_fully_meth_reads', 'ch_fully_unmeth_reads', 'ch_total_reads',
    'cg_fully_meth_reads', 'cg_fully_unmeth_reads', 'cg_total_reads',
    'ch_mhl', 'ch_umhl',
    'cg_mhl', 'cg_umhl',
]


bamfile = pysam.AlignmentFile(input_bam, 'rb')

current_chrom = "chr0"
current_binstart = 0
current_binend = current_binstart + bin_size 
current_binstring = ""

debug_flag = 0

with open(output_mcinfo, 'w') as output_fh:
    # title
    output_fh.write("\t".join(columns)+'\n')
    for i, read in enumerate(bamfile.fetch()):
        if i % verbose_level == 0:
            print(i, time.time()-ti)
            
        chrom, start, end, mc_string = (read.reference_name, 
                                        read.reference_start, 
                                        read.reference_end, 
                                        read.tags[2][1],
                                       )
        # handle chr
        if not chrom.startswith('chr'):
            chrom = 'chr'+chrom
        
        # when current read goes beyond the current bin
        # keep updating the current bin until the current read is within the bin 
        while (current_chrom != chrom) or (start > current_binend):
            # flush the current bin out
            if current_binstring:
                ta = time.time()
                # compute all scores
                mch_str, mcg_str = compute_scores_utils.process_strings(current_binstring)
#                 print(1, time.time()-ta)
#                 debug_flag = 1
#                 break
                # site level
                ch_mc, ch_c = compute_scores_utils.calc_mc(mch_str, 'h')
                cg_mc, cg_c = compute_scores_utils.calc_mc(mcg_str, 'z')
#                 print(2, time.time()-ta)
                # read level
                ch_fmr, ch_fumr, ch_total_reads = compute_scores_utils.calc_fmr(mch_str, 'h')
                cg_fmr, cg_fumr, cg_total_reads = compute_scores_utils.calc_fmr(mcg_str, 'z')
#                 print(3, time.time()-ta)
                # MHL level
                ch_mhl, ch_umhl = compute_scores_utils.calc_mhl_fast(mch_str, 'h')
                cg_mhl, cg_umhl = compute_scores_utils.calc_mhl_fast(mcg_str, 'z')
#                 print(4, time.time()-ta)
                
                # record values
                column_values = [
                    chrom, start, end,
                    ch_mc, ch_c,
                    cg_mc, cg_c,
                    ch_fmr, ch_fumr, ch_total_reads,
                    cg_fmr, cg_fumr, cg_total_reads,
                    ch_mhl, ch_umhl,
                    cg_mhl, cg_umhl,
                ]
#                 print(5, time.time()-ta)
                # output 
                output_fh.write("\t".join([str(item) for item in column_values])+'\n')
#                 print(6, time.time()-ta)
            
            # move current bin to the next 
            # (move to a new chrom or the next bin within the chrom)
            if current_chrom != chrom:
                current_chrom = chrom
                current_binstart = 0 
                current_binend = bin_size
                current_binstring = ""
            else:
                current_binstart += bin_size
                current_binend += bin_size
                current_binstring = ""

        # move the current read the current bin
        current_binstring = current_binstring + (mc_string+',')
    
        if debug_flag == 1:
            break
    # looped over all reads
    
bamfile.close()
print("Total time: {:.2f}".format(time.time()-ti))

0 0.05504465103149414
1000000 9.764495611190796
2000000 19.120623350143433
3000000 28.420944213867188
4000000 37.44082736968994
5000000 46.53837013244629
6000000 55.718910217285156
7000000 64.79138112068176
8000000 73.87649774551392
9000000 83.32796216011047
10000000 92.79159045219421
11000000 102.1103105545044
12000000 111.38528442382812
13000000 120.72018599510193
14000000 130.38233518600464
15000000 139.67582941055298
16000000 149.25942301750183
17000000 159.83411073684692
18000000 169.17061400413513
19000000 178.76193284988403
20000000 188.0892882347107
21000000 197.5148627758026
22000000 206.82871389389038
23000000 216.56706833839417
24000000 226.01077485084534
25000000 235.04964780807495
26000000 244.37783002853394
27000000 253.85458612442017
28000000 263.35264801979065
29000000 272.9853904247284
30000000 282.6707146167755
31000000 292.2439064979553
32000000 301.8628854751587
33000000 311.57815504074097
34000000 320.91417717933655
35000000 330.198543548584
36000000 339.7459542751

### Prototype 5

In [292]:
# prototype5 - implement read split; it agrees with the other method; validated improved time 

# Todo
# wrap this up and allow multiprocessing
"""
Requirements:
- sorted bam file
- bam file created by Bismark (has the "XM" tag in the third position of tags)
- bin_size > sequence length
"""

ti = time.time()

input_bam = '/cndd2/fangming/projects/cfdna/all_data/bam/chan2013_CTR101.deduplicated.sorted.bam'
output_mcinfo = '/cndd2/fangming/projects/cfdna/all_data/bam/test_phase5.mc_scores.txt'
bin_size = 100000 # 100,000
verbose_level = 1000000 # update every 1 million reads

columns = [
    'chr', 'start', 'end',
    'ch_mc', 'ch_c',
    'cg_mc', 'cg_c',
    'ch_fully_meth_reads', 'ch_fully_unmeth_reads', 'ch_total_reads',
    'cg_fully_meth_reads', 'cg_fully_unmeth_reads', 'cg_total_reads',
    'ch_mhl', 'ch_umhl',
    'cg_mhl', 'cg_umhl',
]


bamfile = pysam.AlignmentFile(input_bam, 'rb')
assert bamfile.has_index()

current_chrom = "chr0"
current_binstart = 0
current_binend = current_binstart + bin_size 
current_binstring = ""
next_binstring_hold = "" 


with open(output_mcinfo, 'w') as output_fh:
    # title
    output_fh.write("\t".join(columns)+'\n')
    for i, read in enumerate(bamfile.fetch()):
        if i % verbose_level == 0:
            print(i, time.time()-ti)
        
        # deal with one read
        chrom, start, end, mc_string = (read.reference_name, 
                                        read.reference_start, 
                                        read.reference_end, 
                                        read.tags[2][1],
                                       )
        # handle chr
        if not chrom.startswith('chr'):
            chrom = 'chr'+chrom
        
        # when current read goes beyond the current bin
        # keep updating the current bin until the current read is within the bin 
        while (current_chrom != chrom) or (start > current_binend):
            # flush the current bin out
            if current_binstring:
                # compute all scores
                mch_str, mcg_str = compute_scores_utils.process_strings(current_binstring)
                # site level
                ch_mc, ch_c = compute_scores_utils.calc_mc(mch_str, 'h')
                cg_mc, cg_c = compute_scores_utils.calc_mc(mcg_str, 'z')
                # read level
                ch_fmr, ch_fumr, ch_total_reads = compute_scores_utils.calc_fmr(mch_str, 'h')
                cg_fmr, cg_fumr, cg_total_reads = compute_scores_utils.calc_fmr(mcg_str, 'z')
                # MHL level
                ch_mhl, ch_umhl = compute_scores_utils.calc_mhl_fast(mch_str, 'h')
                cg_mhl, cg_umhl = compute_scores_utils.calc_mhl_fast(mcg_str, 'z')
                
                # record values
                column_values = [
                    current_chrom, current_binstart, current_binend,
                    ch_mc, ch_c,
                    cg_mc, cg_c,
                    ch_fmr, ch_fumr, ch_total_reads,
                    cg_fmr, cg_fumr, cg_total_reads,
                    ch_mhl, ch_umhl,
                    cg_mhl, cg_umhl,
                ]
                # output 
                output_fh.write("\t".join([str(item) for item in column_values])+'\n')
            
            # move current bin to the next 
            # (move to a new chrom or the next bin within the chrom)
            if current_chrom != chrom:
                current_chrom = chrom
                current_binstart = 0 
                current_binend = bin_size
                current_binstring = next_binstring_hold
                next_binstring_hold = ""
            else:
                current_binstart += bin_size
                current_binend += bin_size
                current_binstring = next_binstring_hold
                next_binstring_hold = ""

        # this means current_chrom == chrom, and start <= current bin
        if end <= current_binend:
            # move the whole read into the current bin
            current_binstring += (mc_string+',')
        else:
            # move the first section of the read into the current bin
            current_binstring += (mc_string[:-(end-current_binend)]+',')
            # hold the second section (beyond the current bin) temporarily
            next_binstring_hold += (mc_string[(current_binend-start):]+',')
        
    # looped over all reads
    
bamfile.close()
print("Total time: {:.2f}".format(time.time()-ti))

0 0.08773040771484375
1000000 9.786346912384033
2000000 19.08590340614319
3000000 28.39142084121704
4000000 37.44027280807495
5000000 46.56340980529785
6000000 55.67896127700806
7000000 64.73208141326904
8000000 73.82135653495789
9000000 83.06259155273438
10000000 92.20715117454529
11000000 101.31887459754944
12000000 110.4325065612793
13000000 119.61133909225464
14000000 129.3441445827484
15000000 138.52730321884155
16000000 147.93875241279602
17000000 157.04643058776855
18000000 166.19881129264832
19000000 175.47222876548767
20000000 184.70955657958984
21000000 193.88530087471008
22000000 202.9944236278534
23000000 212.35834622383118
24000000 221.55323195457458
25000000 230.473778963089
26000000 239.5750217437744
27000000 248.70607614517212
28000000 257.84019589424133
29000000 267.2569799423218
30000000 276.642555475235
31000000 285.8227117061615
32000000 295.09400177001953
33000000 304.83662366867065
34000000 314.2331233024597
35000000 323.42360663414
36000000 332.9594056606293
3700

### Prototype 6

In [294]:
# prototype6 -  wrap prototype5 up and allow multiprocessing


input_bam = '/cndd2/fangming/projects/cfdna/all_data/bam/chan2013_CTR101.deduplicated.sorted.bam'
output_scores = '/cndd2/fangming/projects/cfdna/all_data/bam/test_phase5.mc_scores.txt'

def calculate_read_level_mc_metrics_genomewide(
    input_bam, output_scores, 
    bin_size=1000, # 1kb
    verbose_level=1000000 # update every 1 million reads
    ):
    
    """
    Requirements:
    - sorted bam file
    - bam file created by Bismark (has the "XM" tag in the third position of tags)
    - bin_size > sequence length
    """
    logging.info("input: {}".format(input_bam))
    logging.info("output: {}".format(output_scores))
    
    ti = time.time()
    columns = [
        'chr', 'start', 'end',
        'ch_mc', 'ch_c',
        'cg_mc', 'cg_c',
        'ch_fully_meth_reads', 'ch_fully_unmeth_reads', 'ch_total_reads',
        'cg_fully_meth_reads', 'cg_fully_unmeth_reads', 'cg_total_reads',
        'ch_mhl', 'ch_umhl',
        'cg_mhl', 'cg_umhl',
    ]

    bamfile = pysam.AlignmentFile(input_bam, 'rb')
    assert bamfile.has_index()

    current_chrom = "chr0"
    current_binstart = 0
    current_binend = current_binstart + bin_size 
    current_binstring = ""
    next_binstring_hold = "" 

    with open(output_scores, 'w') as output_fh:
        # title
        output_fh.write("\t".join(columns)+'\n')
        for i, read in enumerate(bamfile.fetch()):
            if i % verbose_level == 0:
                print(i, time.time()-ti)

            # deal with one read
            chrom, start, end, mc_string = (read.reference_name, 
                                            read.reference_start, 
                                            read.reference_end, 
                                            read.tags[2][1],
                                           )
            # handle chr
            if not chrom.startswith('chr'):
                chrom = 'chr'+chrom

            # when current read goes beyond the current bin
            # keep updating the current bin until the current read is within the bin 
            while (current_chrom != chrom) or (start > current_binend):
                # flush the current bin out
                if current_binstring:
                    # compute all scores
                    mch_str, mcg_str = compute_scores_utils.process_strings(current_binstring)
                    # site level
                    ch_mc, ch_c = compute_scores_utils.calc_mc(mch_str, 'h')
                    cg_mc, cg_c = compute_scores_utils.calc_mc(mcg_str, 'z')
                    # read level
                    ch_fmr, ch_fumr, ch_total_reads = compute_scores_utils.calc_fmr(mch_str, 'h')
                    cg_fmr, cg_fumr, cg_total_reads = compute_scores_utils.calc_fmr(mcg_str, 'z')
                    # MHL level
                    ch_mhl, ch_umhl = compute_scores_utils.calc_mhl_fast(mch_str, 'h')
                    cg_mhl, cg_umhl = compute_scores_utils.calc_mhl_fast(mcg_str, 'z')

                    # record values
                    column_values = [
                        current_chrom, current_binstart, current_binend,
                        ch_mc, ch_c,
                        cg_mc, cg_c,
                        ch_fmr, ch_fumr, ch_total_reads,
                        cg_fmr, cg_fumr, cg_total_reads,
                        ch_mhl, ch_umhl,
                        cg_mhl, cg_umhl,
                    ]
                    # output 
                    output_fh.write("\t".join([str(item) for item in column_values])+'\n')

                # move current bin to the next 
                # (move to a new chrom or the next bin within the chrom)
                if current_chrom != chrom:
                    current_chrom = chrom
                    current_binstart = 0 
                    current_binend = bin_size
                    current_binstring = next_binstring_hold
                    next_binstring_hold = ""
                else:
                    current_binstart += bin_size
                    current_binend += bin_size
                    current_binstring = next_binstring_hold
                    next_binstring_hold = ""

            # this means current_chrom == chrom, and start <= current bin
            if end <= current_binend:
                # move the whole read into the current bin
                current_binstring += (mc_string+',')
            else:
                # move the first section of the read into the current bin
                current_binstring += (mc_string[:-(end-current_binend)]+',')
                # hold the second section (beyond the current bin) temporarily
                next_binstring_hold += (mc_string[(current_binend-start):]+',')

        # looped over all reads
    bamfile.close()
    logging.info("Total time: {:.2f} ({})".format(time.time()-ti, input_bam))
    
    return

In [310]:
inputs = [
    '/cndd2/fangming/projects/cfdna/all_data/bam/chan2013_CTR101.deduplicated.sorted.bam',
    '/cndd2/fangming/projects/cfdna/all_data/bam/chan2013_CTR102.deduplicated.sorted.bam',
    '/cndd2/fangming/projects/cfdna/all_data/bam/chan2013_CTR103.deduplicated.sorted.bam',
]
outputs = [
    '/cndd2/fangming/projects/cfdna/all_data/bam/test_phase6_101.mc_scores.txt',
    '/cndd2/fangming/projects/cfdna/all_data/bam/test_phase6_102.mc_scores.txt',
    '/cndd2/fangming/projects/cfdna/all_data/bam/test_phase6_103.mc_scores.txt',
]
bin_size = 1000
verbose_level = 1000000
ncores = 4

logger = snmcseq_utils.create_logger()

with Pool(min(ncores, len(inputs))) as p:
#     # test 1 
#     calculate_read_level_mc_metrics_genomewide(
#                         _input, _output, 
#                         bin_size=bin_size,
#                         verbose_level=verbose_level,
#                         )
    # parallel
    p.starmap(calculate_read_level_mc_metrics_genomewide, 
                  [(_input, _output, bin_size, verbose_level,) 
                       for _input, _output in zip(inputs, outputs)],
             )

07/21/2020 04:45:13 PM input: /cndd2/fangming/projects/cfdna/all_data/bam/chan2013_CTR101.deduplicated.sorted.bam
07/21/2020 04:45:13 PM input: /cndd2/fangming/projects/cfdna/all_data/bam/chan2013_CTR103.deduplicated.sorted.bam
07/21/2020 04:45:13 PM input: /cndd2/fangming/projects/cfdna/all_data/bam/chan2013_CTR102.deduplicated.sorted.bam
07/21/2020 04:45:13 PM output: /cndd2/fangming/projects/cfdna/all_data/bam/test_phase6_101.mc_scores.txt
07/21/2020 04:45:13 PM output: /cndd2/fangming/projects/cfdna/all_data/bam/test_phase6_102.mc_scores.txt
07/21/2020 04:45:13 PM output: /cndd2/fangming/projects/cfdna/all_data/bam/test_phase6_103.mc_scores.txt


0 0.05310487747192383
0 0.2539255619049072
1000000 14.299242734909058
1000000 17.551695108413696
2000000 28.751566410064697
2000000 36.91200566291809
3000000 44.21672749519348
3000000 59.223729848861694
4000000 61.466527700424194
5000000 78.10306429862976
4000000 79.77050495147705
6000000 93.91647696495056
5000000 100.3066918849945
7000000 110.27059483528137
6000000 120.4522705078125
8000000 125.86325883865356
7000000 140.63649606704712
9000000 141.81236147880554
10000000 157.93165063858032
8000000 161.35581469535828
11000000 174.3900351524353
9000000 181.30642461776733
12000000 189.81959176063538
10000000 199.76794719696045
13000000 204.96493887901306
14000000 219.38140416145325
11000000 221.22096252441406
15000000 235.92458987236023
12000000 240.40980863571167
16000000 250.96792936325073
13000000 261.0703184604645
17000000 267.3331024646759
14000000 281.09987592697144
18000000 282.78934478759766
19000000 298.0670952796936
15000000 302.3333160877228
20000000 315.2036588191986
16000000

Process ForkPoolWorker-24:
Process ForkPoolWorker-25:
Process ForkPoolWorker-23:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/cndd/fangming/venvs/conda_dobro/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/cndd/fangming/venvs/conda_dobro/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/cndd/fangming/venvs/conda_dobro/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/cndd/fangming/venvs/conda_dobro/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/cndd/fangming/venvs/conda_dobro/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/cndd/fangming/venvs/conda_dobro/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File 

KeyboardInterrupt: 

# Test different Pool 

In [327]:
def test(x):
    print(x**2)
    return x**2


# for i in range(10):
#     print("round {}".format(i))
#     with Pool(4) as p:
#         test_cases = np.arange(5)
#         results = p.map(test, test_cases)
# #         results = p.map(test, test_cases)
#         res_async = p.map_async(test, test_cases, callback=results_async)
        
#         print(results, results_async)
    
with Pool(4) as p:
    test_cases = np.arange(5)
#     results = p.map(test, test_cases)
#     res_async = p.map_async(test, test_cases, callback=results_async)
    for test_case in test_cases:
        result = p.apply(test, (test_case,))
    
print(results, results_async)
    

0
1
4
9
16
[0, 1, 4, 9, 16] <multiprocessing.pool.MapResult object at 0x7f4a76dcdd30>
